In [1]:
!pip install -q findspark
!pip install pyspark
!pip install py4j
!pip install --upgrade py4j

  Using cached py4j-0.10.9.9-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached py4j-0.10.9.9-py2.py3-none-any.whl (203 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyspark 3.5.4 requires py4j==0.10.9.7, but you have py4j 0.10.9.9 which is incompatible.


In [2]:
import json, requests
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, array, struct, lit, get_json_object
import json
import findspark

In [3]:
API_KEY = '14gfkN3hD26VRnFO5CXk3WpUTXKETxdSdY4FjMEL'
START_DATE = '2024-12-01'
END_DATE = '2024-12-5'
url = 'https://api.nasa.gov/neo/rest/v1/feed?start_date={}&end_date={}&api_key={}'.format(START_DATE, END_DATE, API_KEY)

In [4]:
findspark.init()
findspark.find()
response = requests.get(url)
result = response.json()
result

{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-12-5&end_date=2024-12-09&detailed=false&api_key=14gfkN3hD26VRnFO5CXk3WpUTXKETxdSdY4FjMEL',
  'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-11-27&end_date=2024-12-01&detailed=false&api_key=14gfkN3hD26VRnFO5CXk3WpUTXKETxdSdY4FjMEL',
  'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-12-01&end_date=2024-12-5&detailed=false&api_key=14gfkN3hD26VRnFO5CXk3WpUTXKETxdSdY4FjMEL'},
 'element_count': 106,
 'near_earth_objects': {'2024-12-05': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/3442841?api_key=14gfkN3hD26VRnFO5CXk3WpUTXKETxdSdY4FjMEL'},
    'id': '3442841',
    'neo_reference_id': '3442841',
    'name': '(2008 YS27)',
    'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3442841',
    'absolute_magnitude_h': 21.23,
    'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.1508533561,
      'estimated_diameter_max': 0.3373183589},
     'met

In [5]:
spark= SparkSession \
       .builder \
       .appName("asteroid data") \
       .getOrCreate()

In [6]:
neo_list = []
for date, neos in result['near_earth_objects'].items():
    for neo in neos:
        miss_distance_miles = neo['close_approach_data'][0]['miss_distance']['miles']
        neo_list.append((neo['id'], neo['name'], miss_distance_miles))


df = spark.createDataFrame(neo_list, ["id", "name", "miss_distance_miles"])


df.show()

+--------+-----------+-------------------+
|      id|       name|miss_distance_miles|
+--------+-----------+-------------------+
| 3442841|(2008 YS27)| 10813531.994023308|
| 3460325| (2009 KY1)|14046699.5700268952|
| 3656409|(2014 AA53)|17402930.8542236912|
| 3789470| (2017 WL1)| 10202285.614528609|
|54213628| (2021 UQ4)| 32733412.069450866|
|54225710| (2021 WA5)|  2335547.532105324|
|54229471| (2021 XN4)|   4189209.09836505|
|54340322| (2023 BJ7)|15844787.7028767138|
|54354403| (2023 GJ2)| 43384262.804562119|
|54500031| (2024 WB1)| 4553595.0398739572|
|54500315| (2024 WT1)|  5691660.437083179|
|54500320| (2024 WY1)| 7395539.1619332816|
|54501473| (2024 WG5)| 2842485.4873778878|
|54502663|  (2024 XO)| 12504393.271461396|
|54502873| (2024 XD1)| 14284244.348958591|
|54503173|(2024 WA22)| 1348024.2951026172|
|54503203| (2024 XW3)| 3980841.9849891792|
|54503839| (2024 XV4)| 11480895.049953759|
|54506447|(2024 XO15)| 1653294.8244653736|
|54506634|(2024 XB17)|  8552903.355711259|
+--------+-